# Import des modules nécessaires

In [1]:
# Pour manipuler efficacement des tables de données dans Python
import pandas as pd

# Pour faire des requêtes GET et POST
import requests

In [2]:
print(pd.__version__)

0.22.0


In [3]:
print(requests.__version__)

2.18.4


# Requêtage d'une API REST

Pour lancer une requête GET, c'est ultra-simple : on utilise `requests.get(url)`.

Si vous voulez glisser des paramètres avec la requête, utilisez l'argument optionnel `params`, qui accepte un simple dictionnaire.

In [4]:
r = requests.get("https://data.angers.fr/api/records/1.0/search/",
                 params={
                     'dataset': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
                     'rows': 10
                 })

Regardons la nature de ce qui nous est renvoyé.

In [5]:
type(r)

requests.models.Response

C'est un objet de la classe `Response`. Si vous voulez connaitre toutes les méthodes attachées, faites `help(r)`

In [6]:
help(r)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

Pour tester que tout a bien fonctionnné, on va vérifié le statut.

In [7]:
r.raise_for_status()

Si ça dit rien c'est que c'est bon :) Dans le doute, regardons le statut...

In [8]:
r.status_code

200

Statut 200 = succès !!

Pour récupérer le contenu de la réponse, vous pouvez utiliser `r.text`.

Il y a aussi un décodeur JSON, si on sait que la réponse est dans ce format. Du coup, il parse le JSON et rend un dictionnaire !

In [9]:
r.json()

{'nhits': 2345,
 'parameters': {'dataset': ['horaires-theoriques-et-arrets-du-reseau-irigo-gtfs'],
  'format': 'json',
  'rows': 10,
  'timezone': 'UTC'},
 'records': [{'datasetid': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
   'fields': {'location_type': '0',
    'stop_code': '5009',
    'stop_coordinates': [47.51151, -0.59615],
    'stop_id': '1AARD',
    'stop_name': 'AVRILLE - ARDENNE'},
   'geometry': {'coordinates': [-0.59615, 47.51151], 'type': 'Point'},
   'record_timestamp': '2018-06-05T08:42:29+00:00',
   'recordid': '5bd0b33b1cebfc78f65c4a9b3f7c3bc2b694fad2'},
  {'datasetid': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
   'fields': {'location_type': '0',
    'stop_code': '4827',
    'stop_coordinates': [47.508797, -0.591643],
    'stop_id': '1BASC',
    'stop_name': 'BASCULE'},
   'geometry': {'coordinates': [-0.591643, 47.508797], 'type': 'Point'},
   'record_timestamp': '2018-06-05T08:42:29+00:00',
   'recordid': '81f22ac7f1433f2bc9397eb11095b88f608bdb

# Remplissage de DataFrame

On peut construire une `DataFrame` directement à partir d'un "dictionnaires de colonnes".

In [10]:
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [3, 4, 1]
})

df

,a,b
0,1,3
1,2,4
2,3,1


Par exemple, listons les arrêts, leur id, leur nom et leurs coordonnées GPS.

In [11]:
dd = r.json()['records']

In [12]:
dd[0]

{'datasetid': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
 'fields': {'location_type': '0',
  'stop_code': '5009',
  'stop_coordinates': [47.51151, -0.59615],
  'stop_id': '1AARD',
  'stop_name': 'AVRILLE - ARDENNE'},
 'geometry': {'coordinates': [-0.59615, 47.51151], 'type': 'Point'},
 'record_timestamp': '2018-06-05T08:42:29+00:00',
 'recordid': '5bd0b33b1cebfc78f65c4a9b3f7c3bc2b694fad2'}

In [15]:
ids = [elem['fields']['stop_id'] for elem in dd]
names = [elem['fields']['stop_name'] for elem in dd]
lats = [elem['fields']['stop_coordinates'][0] for elem in dd]
lons = [elem['fields']['stop_coordinates'][1] for elem in dd]

df = pd.DataFrame({
    'id': ids,
    'name': names,
    'latitude': lats,
    'longitude': lons
})

df

C:\Users\Puppybutthole\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  del sys.path[0]
